In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [2]:
labels = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
        }

df = pd.read_csv("train.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print(*[(col, "--", df[col].isnull().sum()) for col in df.columns if df[col].isnull().sum() > 0], sep = "\n")

In [3]:
X_cols = df.columns[1: ]
Y_col = df.columns[0]

In [4]:
from sklearn.preprocessing import normalize

X_normalize = pd.DataFrame(normalize(df[X_cols]))

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(df[X_cols], df[Y_col], random_state=81, test_size=0.02)

In [6]:
X_train = np.reshape(np.array(X_train, dtype=np.float64), (-1, 28, 28, 1))
# Y_train = tf.keras.utils.to_categorical(Y_train, 10)

X_test = np.reshape(np.array(X_test, dtype=np.float64), (-1, 28, 28, 1))
# Y_test = tf.keras.utils.to_categorical(Y_test, 10)


X_train /= 255
X_test /= 255

In [17]:
model = tf.keras.models.Sequential()

# model.add(tf.keras.layers.RandomRotation(0.2))
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), input_shape=(28, 28, 1)))
model.add(tf.keras.layers.BatchNormalization(axis=1))
model.add(tf.keras.layers.Softmax())
model.add(tf.keras.layers.MaxPool2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), input_shape=(28, 28, 1)))
model.add(tf.keras.layers.BatchNormalization(axis=1))
model.add(tf.keras.layers.Softmax())
model.add(tf.keras.layers.MaxPool2D((2, 2)))
model.add(tf.keras.layers.Flatten())    
model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer= tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy', ]) 
              # loss="categorical_crossentropy", metrics=['accuracy', ])
model.fit(X_train, Y_train, batch_size= 256, epochs= 10, validation_data=(X_test, Y_test))

Epoch 1/10
230/230 [==============================] - 115s 494ms/step - loss: 0.8343 - sparse_categorical_accuracy: 0.7162 - val_loss: 2.4962 - val_sparse_categorical_accuracy: 0.1167
Epoch 2/10
230/230 [==============================] - 121s 529ms/step - loss: 0.4317 - sparse_categorical_accuracy: 0.8461 - val_loss: 1.1995 - val_sparse_categorical_accuracy: 0.5658
Epoch 3/10
230/230 [==============================] - 147s 637ms/step - loss: 0.3728 - sparse_categorical_accuracy: 0.8646 - val_loss: 0.3888 - val_sparse_categorical_accuracy: 0.8692
Epoch 4/10
230/230 [==============================] - 141s 612ms/step - loss: 0.3430 - sparse_categorical_accuracy: 0.8756 - val_loss: 0.3490 - val_sparse_categorical_accuracy: 0.8842
Epoch 5/10
230/230 [==============================] - 155s 677ms/step - loss: 0.3195 - sparse_categorical_accuracy: 0.8851 - val_loss: 0.3345 - val_sparse_categorical_accuracy: 0.8858
Epoch 6/10
230/230 [==============================] - 115s 502ms/step - loss: 0.

In [14]:
test = pd.read_csv("test.csv")

test_x = np.array(np.reshape(np.array(test, dtype=np.float64), (-1, 28, 28, 1)))
test_x /= 255

test_y = model.predict(test_x)

313/313 [==============================] - 5s 15ms/step


In [15]:
with open("sub.csv", "w") as f :
    
    f.write("ImageId,label\n")
    k = 0

    for i in test_y:
        k += 1
        x = (np.where(i == np.max(i)))[0]
        f.write(str(k) + "," + str(int(x)) + "\n")


In [19]:
df = pd.read_csv("train.csv")

In [20]:
for col in df.columns[1: ]:
    df[col] /= 255

X_train, X_test, Y_train, Y_test = train_test_split(df[X_cols], df[Y_col], random_state=81, test_size=0.1)

In [27]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

svm = make_pipeline(StandardScaler(), SVC(random_state= 45, decision_function_shape='ovo', C=10))
svm.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc',
                 SVC(C=100, decision_function_shape='ovo', random_state=45))])

In [28]:
from sklearn.metrics import accuracy_score

print(accuracy_score(Y_test, svm.predict(X_test)))

0.896


In [29]:
test = pd.read_csv("test.csv")

for col in test.columns : 
    test[col] /= 255

test['label'] = svm.predict(test)

print(test['label'])

0       0
1       1
2       2
3       2
4       3
       ..
9995    0
9996    6
9997    8
9998    8
9999    1
Name: label, Length: 10000, dtype: int64


In [31]:
with open('sub.csv', 'w+') as f :

    f.write("ImageId" + "," + "label" + "\n")
     
    k = 0
    for i in test['label'] :
        k += 1
        f.write(str(k) + "," + str(i) + "\n")